In [1]:
import geopandas as gpd
import pandas as pd
import seaborn as sns
import traffic
import numpy as np
import pyproj
from scipy.spatial.distance import cdist

In [4]:
# airprox_gdf = gpd.GeoDataFrame(pd.read_pickle('../data/airprox_asp_2000_2021.pkl'))
# tfc_clean = traffic.core.Traffic.from_file('../data/cornwall_tfc_clean_30s_lt3000ft_2019_filt10.pkl')
tfc_clean = traffic.core.Traffic.from_file('../data/soton_tfc_clean_lt5000ft_201901.pkl')

AttributeError: module 'traffic' has no attribute 'core'

In [5]:
tfc_clean.data.head()

NameError: name 'tfc_clean' is not defined

In [ ]:
tfc_agg = tfc_clean.compute_xy('epsg:3857').assign(
    x=lambda elt: (elt.x // 4000) * 4000,
    y=lambda elt: (elt.y // 4000) * 4000,
).groupby(["x", "y"]).agg(altitude_mean=pd.NamedAgg('altitude', np.mean),
                          altitude_std=pd.NamedAgg('altitude', np.std), track_mean=pd.NamedAgg('track', np.mean),
                          track_std=pd.NamedAgg('track', np.std), groundspeed_mean=pd.NamedAgg('groundspeed', np.mean),
                          groundspeed_std=pd.NamedAgg('groundspeed', np.std), flight_id_nunique=('flight_id', 'nunique'))

In [ ]:
tfc_agg.head(10)

In [ ]:
# tfc_agg.to_pickle('../data/cornwall_tfc_clean_30s_lt3000ft_2019_filt10_aggxy.pkl')

In [ ]:
# airprox_gdf = gpd.GeoDataFrame(pd.read_pickle('../data/airprox_asp_2000_2021.pkl'))
# tfc_agg = pd.read_pickle('../data/cornwall_tfc_clean_30s_lt3000ft_2019_filt10_aggxy.pkl')
pd.__version__

In [ ]:
airprox_gdf.head()

In [ ]:
x_idx = np.array(tfc_agg.index.levels[0])
y_idx = np.array(tfc_agg.index.levels[1])


In [ ]:
airprox_gdf = airprox_gdf[
    (airprox_gdf.Latitude >= tfc_clean.data.latitude.min()) &
    (airprox_gdf.Latitude <= tfc_clean.data.latitude.max()) &
    (airprox_gdf.Longitude >= tfc_clean.data.longitude.min()) &
    (airprox_gdf.Longitude <= tfc_clean.data.longitude.max())]

In [ ]:
tfc_proj = pyproj.Proj(
    proj="lcc",
    ellps="WGS84",
    lat_1=tfc_clean.data.latitude.min(),
    lat_2=tfc_clean.data.latitude.max(),
    lat_0=tfc_clean.data.latitude.mean(),
    lon_0=tfc_clean.data.longitude.mean(),
)
transformer = pyproj.Transformer.from_proj(pyproj.Proj("epsg:4326"), tfc_proj, always_xy=True)
x, y = transformer.transform(
    airprox_gdf.Longitude.values,
    airprox_gdf.Latitude.values,
)
airprox_gdf = airprox_gdf.assign(x=x, y=y)

In [ ]:
airprox_gdf

In [ ]:
tfc_grid = np.array(tfc_agg.reset_index()[['x', 'y']])
airprox_locs = np.array(airprox_gdf[['x', 'y']])

In [ ]:
tfc_idxs = cdist(tfc_grid, airprox_locs).argmin(axis=0)

In [ ]:
tfc_cells = tfc_agg.iloc[tfc_idxs]
airproxes_with_tfc = pd.concat([airprox_gdf.reset_index(), tfc_cells.reset_index()], axis=1)
airproxes_with_tfc = airproxes_with_tfc.drop(labels=['index', 'index_right', 'x', 'y', 'country'], axis=1)
airproxes_with_tfc

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('font', size=20)
fig = plt.figure(figsize=(10, 8))
ax = fig.add_axes([0, 0, 6, 6])
axx = fig.add_axes([0, 6.1, 4.8, 1], sharex=ax)
axy = fig.add_axes([6.1, 0, 1, 6], sharey=ax)
tfc_agg.reset_index().plot('x', 'y', kind='hexbin', gridsize=100, bins='log', ax=ax, cmap='inferno')
tfc_agg.reset_index()['x'].hist(ax=axx, bins=32)
tfc_agg.reset_index()['y'].hist(ax=axy, orientation='horizontal', bins=32)

In [ ]:
airproxes_with_tfc.explore('track_mean', cmap='inferno')

In [ ]:
airproxes_with_tfc.dtypes

In [ ]:
# corr = airproxes_with_tfc.corr(method='spearman')
corr = airproxes_with_tfc.apply(lambda x: pd.factorize(x)[0]).corr(method='pearson', min_periods=1)
fig, ax = plt.subplots(figsize=(40, 40))
sns.heatmap(corr, square=True, cmap=sns.color_palette('icefire', as_cmap=True), annot=True, ax=ax)
plt.savefig('corr.svg', dpi=500)

In [ ]:
tfc_agg.describe()